In [8]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from collections import Counter
import time
import sys

In [9]:
import json

hw = pd.read_json("./output/whitelist/huawei.json", orient="index")
hw.head()

,clust,date,description,sourceName,title,url
0,4,2019-03-04,Huawei’s already proven that a ban in the worl...,techcrunch,Huawei reportedly plans to sue US government o...,http://techcrunch.com/2019/03/04/huawei-report...
1,2,2019-03-03,The lawyers for Huawei Chief Financial Officer...,reuters,Huawei CFO suing Canada over December arrest,https://www.reuters.com/article/us-usa-china-h...
2,3,2019-03-06,The surprise arrest of Huawei's chief financia...,reuters,"Long before Trump’s trade war with China, Huaw...",https://www.reuters.com/article/us-usa-china-h...
3,4,2019-03-04,Chinese telecom firm Huawei has recently come ...,gizmodo,Chinese Tech Giant Huawei Plans to Sue the US ...,https://gizmodo.com/chinese-tech-giant-huawei-...
4,0,2019-03-14,The company apparently has backups ready if te...,cnet,Huawei reportedly develops its own OS in case ...,https://www.cnet.com/news/huawei-has-reportedl...


In [27]:
tags = ['trudeau trouble', 'trudeau', 'trouble', 'justin trudeau', 'justin']
finalTags = []
tagWords = []
d = {}
for tag in tags:
    tagSplit = tag.split(' ')
    isUnique = True
    for t in tagSplit:
        if t not in tagWords:
            tagWords.append(t)
        else:
            tagWords.append(t)
            isUnique = False
            for t in tagSplit:
                tagWords.remove(t)
    if isUnique:
        finalTags.append(tag)

finalTags

['trudeau trouble', 'justin']

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix

# Remove Stop[words
def remove_sw(x):
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize

    example_sent = x.lower()
    
    custom_stopwords_list = [
        "new york times", #publication
        "breitbart", #publication
        "briefing",
        "evening",
        "yahoo",
        "news",
        "castanet",
        "the",
        "of",
        "says",
        "cnn",
        "fox",
        "bbc",
        "also",
        "said",
        "mr",
        "mrs",
        "im",
        "would",
        "could",
        "should",
        "first",
        "like",
        "dont",
        "wont",
        "get",
        "going"
    ]
    for x in custom_stopwords_list : 
        example_sent = example_sent.replace(x, "")
    
    stop_words = set(stopwords.words('english'))
    stop_words = [x.lower() for x in stop_words]
    word_tokens = word_tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return " ".join(filtered_sentence)

def get_top_n_bigrams(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    #vec = CountVectorizer(max_df=0.85, ngram_range=(1,3)).fit(corpus)
    vectorizer = TfidfVectorizer(ngram_range=(1,3))
    vec = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names()
    sorted_items=sort_coo(vec.tocoo())
    return extract_topn_from_vector(feature_names, sorted_items)

#Function for sorting tf_idf in descending order
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=5):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    # sorted_items = sorted_items[:topn]
    
    finalTags = []
    tagWords = []
    
    for tag in sorted_items:
        tag = feature_names[tag[0]]
        if len(finalTags) != topn:
            tagSplit = tag.split(' ')
            isUnique = True
            for t in tagSplit:
                if t not in tagWords:
                    tagWords.append(t)
                else:
                    isUnique = False
            if isUnique:
                finalTags.append(tag)
        else:
            break
    
    return finalTags

#     score_vals = []
#     feature_vals = []
    
#     # word index and corresponding tf-idf score
#     for idx, score in sorted_items:
        
#         #keep track of feature name and its corresponding score
#         score_vals.append(round(score, 3))
#         feature_vals.append(feature_names[idx])
    
#     return feature_vals
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
#     results= {}
#     for idx in range(len(feature_vals)):
#         results[feature_vals[idx]]=score_vals[idx]
    
#     return results

In [56]:
hw1 = hw.loc[hw['clust'] == 0]
hw1["desc2"] =  hw1["description"].apply(lambda x : remove_sw(x))
hw1["title2"] =  hw1["title"].apply(lambda x : remove_sw(x))
hw1.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,clust,date,description,sourceName,title,url,desc2,title2
4,0,2019-03-14,The company apparently has backups ready if te...,cnet,Huawei reportedly develops its own OS in case ...,https://www.cnet.com/news/huawei-has-reportedl...,company apparently backups ready tensions us g...,huawei reportedly develops os case ca n't use ...
16,0,2019-03-07,The Republican and Democratic leaders of the U...,reuters,U.S. senators offer resolution backing Canada ...,https://www.reuters.com/article/us-usa-china-h...,republican democratic leaders u.s. senate fore...,u.s. senators fer resolution backing canada hu...
20,0,2019-03-07,The Republican and Democratic leaders of the U...,reuters,UPDATE 1-U.S. senators offer resolution backin...,https://www.reuters.com/article/usa-china-huaw...,republican democratic leaders u.s. senate fore...,update 1-u.s. senators fer resolution backing ...
29,0,2019-03-07,The Republican and Democratic leaders of the U...,reuters,U.S. senators offer resolution backing Canada ...,https://in.reuters.com/article/us-usa-china-hu...,republican democratic leaders u.s. senate fore...,u.s. senators fer resolution backing canada hu...
51,0,2019-03-07,The Republican and Democratic leaders of the U...,reuters,U.S. senators offer resolution backing Canada ...,https://ca.reuters.com/article/businessNews/id...,republican democratic leaders u.s. senate fore...,u.s. senators fer resolution backing canada hu...


In [59]:
print(get_top_n_bigrams(list(hw1["title2"])))
print(get_top_n_bigrams(list(hw1["desc2"])))
# print(get_top_n_trigrams(list(hw1["title2"]))[:10])
# print(get_top_n_quadgrams(list(hw1["title2"]))[:10])

['update senators fer', 'huawei cfo', 'reportedly developed os', 'windows cnet']
['ready tensions us', 'wanzhou united states', 'thursday backing canada', 'technologies chief financial', 'senate foreign relations']


In [60]:
hw2 = hw.loc[hw['clust'] == 2]
hw2["title2"] =  hw2["title"].apply(lambda x : remove_sw(x))
hw2["desc2"] =  hw2["description"].apply(lambda x : remove_sw(x))
hw2.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,clust,date,description,sourceName,title,url,title2,desc2
1,2,2019-03-03,The lawyers for Huawei Chief Financial Officer...,reuters,Huawei CFO suing Canada over December arrest,https://www.reuters.com/article/us-usa-china-h...,huawei cfo suing canada december arrest,lawyers huawei chief financial ficer meng wanz...
5,2,2019-03-07,Chinese telecoms equipment maker Huawei Techno...,reuters,Huawei fights back against U.S. blackout with ...,https://www.reuters.com/article/us-usa-china-h...,huawei fights back u.s. blackout texas lawsuit,chinese telecoms equipment maker huawei techno...
8,2,2019-03-04,Huawei Technologies Co. Chief Financial Office...,bloomberg,Huawei CFO Seeks to Undermine Extradition Case...,https://www.bloomberg.com/news/articles/2019-0...,huawei cfo seeks undermine extradition case ci...,huawei technologies co. chief financial ficer ...
9,2,2019-03-07,"Huawei Technologies Co., one of China’s most-g...",bloomberg,How Huawei Became a Target for the U.S. Govern...,https://www.bloomberg.com/news/articles/2019-0...,huawei became tar u.s. government,"huawei technologies co. , one china ’ most-glo..."
14,2,2019-03-07,Huawei fights back against US black out with T...,reuters,Huawei fights back against US black out with T...,https://www.reuters.com/article/test2-usa-chin...,huawei fights back us black texas lawsuit - re...,huawei fights back us black texas lawsuit reut...


In [62]:
print(get_top_n_bigrams(list(hw2["title2"])))
print(get_top_n_bigrams(list(hw2["desc2"])))

['tar government', 'fights back blackout', 'suing canada december', 'setback pr campaign', 'undermine extradition case']
['laptop exceptional', 'setback', 'pr campaign', 'cfo', 'congress']


In [63]:
import json
def getTags(path_to_json, name):

    hw = pd.read_json(path_to_json, orient="index")
    clustTags = {}
    for clust in hw['clust'].unique():
        hwClust = hw.loc[hw['clust'] == clust]
        hwClust["title2"] =  hwClust["title"].apply(lambda x : remove_sw(x))
        clustTags[str(clust)] = get_top_n_bigrams(list(hwClust["title2"]))
    
    jsonName = "./output/tags/whitelist/" + str(name) +'-tags.json'
    with open(jsonName, 'w') as fp:
        json.dump(clustTags, fp)

getTags("./output/whitelist/huawei.json", "huawei")
getTags("./output/whitelist/shutdown.json", "shutdown")
getTags("./output/whitelist/venezuela.json", "venezuela")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind